In [ ]:
import math
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import empyrical as empy
from sklearn.metrics import (
    precision_score, accuracy_score, confusion_matrix, 
    mean_squared_error, recall_score, f1_score
)

import itertools
import random
random.seed(0)

import dataframe_image as dfi
from IPython.display import Image

import warnings
warnings.filterwarnings("ignore")

In [4]:
price_data_name = 'data/backtest_etf_algo_corr_fsp_20220829.csv'
result_excel_name = './result/2022/MPT_kpi/1.5.1b/digital_twins_KPIs_151b_agg.xlsx'
client = 'agg'

comm_ratio = 0.0001
annual_interest = 0.0158
vote_down_buy_ratio_list = [0, 0.1, 0.2, 0.3]

start = '2008'
end = '2022-07'

In [5]:
def display_df(df, export_img=False, **kwargs):
    df_style = df.style.background_gradient(**kwargs)
    display(df_style)
    
    if export_img:
        try:
            dfi.export(df_style,"tmp.png",table_conversion='matplotlib')
            display(Image("tmp.png"))
        except:
            pass

def show_cum_ret(ret):
    fig, ax = plt.subplots(figsize=(24,15))
    cm = plt.get_cmap('gist_rainbow')
    NUM_COLORS = ret.shape[1]
    ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

    (1+ret).cumprod().plot(ax=ax)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_yscale("log")
    plt.show()

In [6]:
def correct_return(ret, factor):
    ret_ = (1+ret)*factor - 1
    return ret_

def add_lev_cost_to_ret(ret, lev, annual_interest=annual_interest):
    
    balance = return_to_price(ret).iloc[1:]
    cash = (1-lev)*balance
    
    """
    balance = hist['Balance']
    balance_ = pd.concat([pd.Series(1e9), balance])
    ret = balance_.pct_change().ffill().dropna()
    """
    
    '''
    monthly_credit = hist['cash'].groupby(pd.Grouper(freq='M')).min()
    idx = hist.index.to_series()
    idx_month_end = idx.groupby(pd.Grouper(freq='M')).max()
    monthly_credit.index = idx_month_end.values
    
    lev_cost = monthly_credit.apply(lambda x: -x*annual_interest/12 if x<0 else 0)
    lev_cost = lev_cost.reindex(balance.index).fillna(0)
    '''
    # cost calculated daily, billed at month end
    daily_credit = cash.apply(lambda x: x if x<0 else 0)
    daily_lev_cost = daily_credit.apply(lambda x: -x*annual_interest/252 if x<0 else 0)
    
    
    #display(daily_lev_cost)
    
    
    monthy_lev_cost = daily_lev_cost.groupby(pd.Grouper(freq='M')).sum()
    
    idx = hist.index.to_series()
    idx_month_end = idx.groupby(pd.Grouper(freq='M')).max()
    
    
    #print(monthy_lev_cost)
    #print(idx_month_end)
    
    
    
    
    #monthy_lev_cost.index = idx_month_end.values
    monthy_lev_cost.index = idx_month_end.reindex(monthy_lev_cost.index).values
    
    
    
    lev_cost = monthy_lev_cost.reindex(balance.index).fillna(0)
    
    lev_cost_correct_factor = 1 - lev_cost/balance
    
    ret_lev_cost_correct = correct_return(ret, lev_cost_correct_factor)
    
    return ret_lev_cost_correct


In [7]:
def drawdown_from_price(price):
    if isinstance(price, pd.Series):
        return _drawdown_from_price(price)
    elif isinstance(price, pd.DataFrame):
        return price.apply(_drawdown_from_price)
    else:
        raise non_pd_error

def _drawdown_from_price(price):
    shift_max = price.copy()
    _max = price.iloc[0]
    for i, j in price.items():
        #print('i', i)
        #print('j', j)
        _max = max(_max, j)
        shift_max[i] = _max
    return price / shift_max - 1

def return_to_price(ret, ini=100):
    price_0 = ret.dropna().iloc[:1] * 0 + ini
    price_0.index = [0]
    price = (1+ret).cumprod() * ini
    return pd.concat([price_0, price])

def drawdown_from_return(ret, ini=100):
    price = return_to_price(ret, ini)
    return drawdown_from_price(price).iloc[1:]

def avg_drawdown(ret):
    dd = drawdown_from_return(ret)
    return dd.mean()

def empy_metric(ret):
    if isinstance(ret, pd.DataFrame):
        return ret.apply(empy_metric).T
    total_return = lambda x: (1+x).prod()-1
    met_func = [
        total_return, 
        lambda x: empy.annual_return(x), 
        lambda x: empy.sharpe_ratio(x), 
        lambda x: empy.annual_volatility(x), 
        lambda x: empy.max_drawdown(x), 
        avg_drawdown]
    
    met_func_names = ['total_return', 'annual_return', 'sharpe_ratio', 'annual_volatility', 
                      'max_drawdown', 'avg_drawdown',]
    
    se = pd.Series([f(ret) for f in met_func], met_func_names)
    
    se['return/maxdd'] = -se.annual_return/se.max_drawdown
    se['return/avgdd'] = -se.annual_return/se.avg_drawdown
    
    #buy01 = ret.apply(lambda x: 0 if x==0 else 1)
    #se['buy_ratio'] = buy01.mean()
    #se['flip_ratio'] = (buy01-buy01.shift()).abs().mean()
    
    return se

#ewma = lambda x, y: x.ewm(span=y).mean()


def cal_metric(y_true, y_pred):

    if isinstance(y_pred, pd.DataFrame):
        prec_ser = pd.Series(index=y_pred.columns, name='precision')
        acccc_ser = pd.Series(index=y_pred.columns, name='accuracy')
        recall_ser = pd.Series(index=y_pred.columns, name='recall')
        recall0_ser = pd.Series(index=y_pred.columns, name='recall0')
        f1_score_ser = pd.Series(index=y_pred.columns, name='f1_score')
        f0_score_ser = pd.Series(index=y_pred.columns, name='f0_score')
        for col in y_pred.columns:
            # print("   *********************************  ", col)
            if isinstance(y_true, pd.DataFrame):
                tmp_y_true = y_true[col]
            else:
                tmp_y_true = y_true.copy()
            prec_ser[col], acccc_ser[col], recall_ser[col], recall0_ser[col], f1_score_ser[col], f0_score_ser[col] = cal_metric(
                tmp_y_true, y_pred[col])
        return prec_ser, acccc_ser, recall_ser, recall0_ser, f1_score_ser, f0_score_ser

    y_true = y_true.reindex(y_pred.index)
    tmp = pd.concat([y_true, y_pred], axis=1).dropna()
    y_true = tmp.iloc[:, 0]
    y_pred = tmp.iloc[:, -1]
    # print("y_pred\n", y_pred)
    # print("y_true\n", y_true)
    # pd.concat([y_proba, y_pred, y_pctch, y_true, ydelta_true], axis=1).to_csv("./tmp/{}.csv".format(y_pctch.name))

    try:
        prec = precision_score(y_true=y_true, y_pred=y_pred)
        acccc = accuracy_score(y_true=y_true, y_pred=y_pred)
    except Exception as e:
        print(e)
        prec = 0
        acccc = 0

    try:
        recall = recall_score(y_true=y_true, y_pred=y_pred)
        recall0 = recall_score(y_true=y_true, y_pred=y_pred, pos_label=0)
    except Exception as e:
        print(e)
        recall = 0
        recall0 = 0

    try:
        f1score = f1_score(y_true=y_true, y_pred=y_pred)
        f0score = f1_score(y_true=y_true, y_pred=y_pred, pos_label=0)
    except Exception as e:
        print(e)
        f1score = 0
        f0score = 0
    # print("mse, mse_proba, msefullscore, msefullscore_ret, prec, acccc, recall, recall0")
    # print(mse, mse_proba, msefullscore, msefullscore_ret, prec, acccc, recall, recall0)
    return prec, acccc, recall, recall0, f1score, f0score

In [8]:
hist = pd.read_excel(result_excel_name, index_col=0, sheet_name='History')
hist.index = pd.to_datetime(hist.index)
balance = hist['Balance']
balance_ = pd.concat([pd.Series(1e9),balance])
ret = balance_.pct_change().ffill().dropna()
ret.index = pd.to_datetime(ret.index)
# ret = ret[start:end]
ret_comm_correct = ret[start:end]
ret_comm_correct

2008-01-02    0.021213
2008-01-03    0.004646
2008-01-04   -0.008659
2008-01-07   -0.003821
2008-01-08    0.002096
                ...   
2022-07-25   -0.000582
2022-07-26   -0.002362
2022-07-27    0.009400
2022-07-28    0.009599
2022-07-29    0.003917
Length: 3670, dtype: float64

In [9]:
close_ratio = hist.iloc[:, 9:]
close_ratio = close_ratio.sort_index(axis=1)
open_not_traded_ratio = close_ratio.shift().fillna(0)
price = pd.read_csv(price_data_name, index_col=0)
price = price.pivot_table('adj_close', 'date', 'code')
price.index = pd.to_datetime(price.index)
price_change = price.pct_change().fillna(0)
price_change = price_change.reindex(close_ratio.index)
open_traded_ratio = close_ratio/(1+price_change)
trade_diff = (open_traded_ratio - open_not_traded_ratio).applymap(abs)
sum_abs_trade_diff = trade_diff.applymap(abs).sum(axis=1)

comm = sum_abs_trade_diff*comm_ratio
comm_correct_factor = 1 - comm/balance

ret_comm_correct = correct_return(ret, comm_correct_factor)
ret_comm_correct = ret_comm_correct[start:end]
ret_comm_correct

2008-01-02    0.021031
2008-01-03    0.004646
2008-01-04   -0.008659
2008-01-07   -0.003824
2008-01-08    0.002096
                ...   
2022-07-25   -0.000582
2022-07-26   -0.002362
2022-07-27    0.009400
2022-07-28    0.009597
2022-07-29    0.003912
Length: 3670, dtype: float64

In [ ]:
preds_folder = 'result/2022/202205v0_till202208/pp10/'

vote_csvs = []
for f in os.listdir(preds_folder):
    if os.path.isdir(os.path.join(preds_folder, f)):
        for ff in os.listdir(os.path.join(preds_folder, f)):
            if ff.startswith('3voteContNdays_'):
                vote_csvs.append(os.path.join(preds_folder, f, ff))

vote_csvs

In [11]:
vote_csvs = [
    './result/2022/cashON_signal/corr_threshold_v1.csv',
    # './result/2022/cashON_signal/corr_threshold_v1-combine2condition.csv',
]

In [13]:
vote_file = './result/2022/cashON_signal/corr_threshold_v1.csv'
vote_df = pd.read_csv(vote_file, index_col=0)
vote_df.index = pd.to_datetime(vote_df.index)
vote_df = vote_df.shift()
vote_freq_cont_df = vote_df[start:end].reindex(ret.index).ffill()

In [14]:
def multi_condition(df, a, b, operation):
    if operation == '&':
        ser = df[a] & df[b]
        ser.name = "({})&({})".format(a, b)
        
    elif operation == '|':
        ser = df[a] | df[b]
        ser.name = "({})|({})".format(a, b)
    else:
        ser = pd.Series()
    return ser

In [16]:
best_idx = './result/2022/cashON_signal/best_case.csv'
pairs = list(pd.read_csv(best_idx, index_col=0).index)
# pairs = list(itertools.combinations(vote_freq_cont_df.columns, 2))
pairs

,0,0.1
(corr_avg__lb15__out-of0.1&0.5)&(corr_avg_pct__lb20__out-of-0.1&0.2),(corr_avg__lb15__out-of0.1&0.5)&(corr_avg_pct_...,(corr_avg__lb15__out-of0.1&0.5)&(corr_avg_pct_...
(corr_avg__lb15__>0.5)&(corr_avg_pct__lb20__out-of-0.1&0.2),(corr_avg__lb15__>0.5)&(corr_avg_pct__lb20__ou...,(corr_avg__lb15__>0.5)&(corr_avg_pct__lb20__ou...
(corr_avg__lb15__>0.5)&(corr_avg__lb50__out-of0.25&0.5),(corr_avg__lb15__>0.5)&(corr_avg__lb50__out-of...,(corr_avg__lb15__>0.5)&(corr_avg__lb50__out-of...
(corr_avg__lb15__out-of0.1&0.5)&(corr_avg__lb50__out-of0.25&0.5),(corr_avg__lb15__out-of0.1&0.5)&(corr_avg__lb5...,(corr_avg__lb15__out-of0.1&0.5)&(corr_avg__lb5...
(corr_avg__lb50__out-of0.25&0.5)&(corr_std__lb10__<0.3),(corr_avg__lb50__out-of0.25&0.5)&(corr_std__lb...,(corr_avg__lb50__out-of0.25&0.5)&(corr_std__lb...
...,...,...
(corr_avg__lb10__<0.1)|(corr_avg_pct__lb10__out-of-0.25&0.75),(corr_avg__lb10__<0.1)|(corr_avg_pct__lb10__ou...,NaN
(corr_std__lb50__out-of0.3&0.6)|(corr_std_abs_pct__lb15__>0.1),(corr_std__lb50__out-of0.3&0.6)|(corr_std_abs_...,NaN
(corr_std_abs__lb20__>0.35)&(corr_std_pct__lb15__<-0.2),(corr_std_abs__lb20__>0.35)&(corr_std_pct__lb1...,NaN
(corr_avg__lb50__out-of0.25&0.45)|(corr_avg_pct__lb20__out-of-0.2&0.4),(corr_avg__lb50__out-of0.25&0.45)|(corr_avg_pc...,NaN


In [12]:
all_vote_down_buy_ratio_list = []
for vote_file in vote_csvs:
    vote_df = pd.read_csv(vote_file, index_col=0)
    vote_df.index = pd.to_datetime(vote_df.index)
    vote_df = vote_df.shift()
    vote_freq_cont_df = vote_df[start:end].reindex(ret.index).ffill()

    vote_down_buy_ratio_df_list = []
    for vote_down_buy_ratio in vote_down_buy_ratio_list:
        v = vote_freq_cont_df.applymap(lambda x: vote_down_buy_ratio if x<0.5 else 1)
        v.columns = v.columns.map(lambda x: x+'___vote_down_buy_{}'.format(vote_down_buy_ratio))
        vote_down_buy_ratio_df_list.append(v)
    vote_down_buy_ratio_df = pd.concat(vote_down_buy_ratio_df_list, 1)
    # vote_down_buy_ratio_df.to_csv(vote_down_buy_ratio_csv)
    all_vote_down_buy_ratio_list.append(vote_down_buy_ratio_df)
all_vote_down_buy_ratio_df = pd.concat(all_vote_down_buy_ratio_list, 1)
all_vote_down_buy_ratio_df

MemoryError: Unable to allocate 1.24 GiB for an array with shape (45487, 3670) and data type float64

In [ ]:
vote_down_buy_ratio_csv = './result/2022/cashON_signal/vote_down_buy_ratio_corr_threshold_v1-sliding.csv'

In [ ]:
all_vote_down_buy_ratio_df = pd.read_csv(vote_down_buy_ratio_csv, index_col=0, usecols=list(range(0, 45493)))
all_vote_down_buy_ratio_df.index = pd.to_datetime(all_vote_down_buy_ratio_df.index)
all_vote_down_buy_ratio_df = all_vote_down_buy_ratio_df.shift()
all_vote_down_buy_ratio_df = all_vote_down_buy_ratio_df.reindex(ret_comm_correct.index)
display(all_vote_down_buy_ratio_df.shape)

In [ ]:
# filter all 1 case
sum_vote_down_buy_ratio_df = all_vote_down_buy_ratio_df.sum()[(all_vote_down_buy_ratio_df.sum()/all_vote_down_buy_ratio_df.count()!=1)]
all_vote_down_buy_ratio_df = all_vote_down_buy_ratio_df[sum_vote_down_buy_ratio_df.index]
display(all_vote_down_buy_ratio_df.shape)

In [ ]:
no_vote = (all_vote_down_buy_ratio_df.iloc[:,0]*0+1).rename('no_3vote')
vote_all_kinds = pd.concat([no_vote, all_vote_down_buy_ratio_df], axis=1)
vote_all_kinds

In [ ]:
vote_shift = vote_all_kinds.shift()
vote_shift.iloc[0] = vote_all_kinds.iloc[0]
flip = (vote_all_kinds - vote_shift).abs()
flip.sum()

In [ ]:
vote_all_kinds_return = vote_all_kinds.apply(lambda x: x*ret_comm_correct)
vote_all_kinds_return.head()

In [ ]:
lev = (hist['asset'] / balance)[start:end]
lev.mean()

In [ ]:
vote_comm_correct_factor = 1 - flip.apply(lambda x: x*lev)*comm_ratio
vote_comm_correct_factor.prod()

In [ ]:
vote_all_kinds_return_correct_comm = correct_return(vote_all_kinds_return, vote_comm_correct_factor)
vote_all_kinds_return_correct_comm

In [ ]:
lev_all = vote_all_kinds.apply(lambda x: x*lev)
lev_all

In [ ]:
vote_all_kinds_return_correct_comm_levc = pd.concat([
    add_lev_cost_to_ret(vote_all_kinds_return_correct_comm[x], lev_all[x], annual_interest) for x in vote_all_kinds_return_correct_comm.columns],1)
vote_all_kinds_return_correct_comm_levc

In [ ]:
vote_all_kinds_return_correct_comm_levc.to_csv('result/2022/portfolioY/ret_corr_threshold_v1-combine2condition(vote_down0).csv')

In [ ]:
# 'result/2022/202205v0_till202208/pp10/vote-pp10-Top10_byAccuracy-2022-Aug-24\\3voteContNdays_202205v0_till202208_Top10_byAccuracy.csv'
# vote_all_kinds_return_correct_comm_csv = 'result/2022/202205v0_till202208/pp10/met_vote_ml_mpt151b_all_kinds_ext_{}.csv'.format(client)
vote_all_kinds_return_correct_comm_csv = 'result/2022/portfolioY/met_corr_threshold_v1-combine2condition(vote_down0).csv'

In [ ]:
try:
    met = pd.read_csv(vote_all_kinds_return_correct_comm_csv, index_col=0)
except:
    met = empy_metric(vote_all_kinds_return_correct_comm_levc)

    met['up_ratio'] = vote_all_kinds.applymap(lambda x: 0 if x<0.99 else 1).mean()
    met['buy_ratio'] = vote_all_kinds.mean()
    met['flip_ratio'] = flip.mean()
    
    """
    met['score'] = met.annual_return * met.sharpe_ratio
    score_excess1 = met.annual_return.apply(lambda x: max(0, x/met.loc['no_3vote','annual_return'] - 1))
    score_excess2 = met.sharpe_ratio.apply(lambda x: max(0, x/met.loc['no_3vote','sharpe_ratio'] - 1))
    score_excess3 = met['return/avgdd'].apply(lambda x: max(0, x/met.loc['no_3vote','return/avgdd'] - 1))
    met['score_excess'] = score_excess1*score_excess2*score_excess3
    met['score_excess_alt'] = score_excess2*score_excess3
    """
    
    
    # met['score_123'] = met.annual_return * met.sharpe_ratio * met['return/avgdd']
    # met['score_12'] = met.annual_return * met.sharpe_ratio
    # met['score_23'] = met.sharpe_ratio * met['return/avgdd']


    # score_excess1 = met.annual_return.apply(lambda x: max(0, x/met.loc['no_3vote','annual_return'] - 1))
    # score_excess2 = met.sharpe_ratio.apply(lambda x: max(0, x/met.loc['no_3vote','sharpe_ratio'] - 1))
    # score_excess3 = met['return/avgdd'].apply(lambda x: max(0, x/met.loc['no_3vote','return/avgdd'] - 1))

    # met['score_excess_123'] = score_excess1*score_excess2*score_excess3
    # met['score_excess_12'] = score_excess1*score_excess2
    # met['score_excess_23'] = score_excess2*score_excess3

    
    
    
    met.to_csv(vote_all_kinds_return_correct_comm_csv)

met

In [ ]:
met = pd.read_csv(vote_all_kinds_return_correct_comm_csv, index_col=0)

In [ ]:
show_metrics = [
    'total_return', 'annual_return', 'sharpe_ratio', 'annual_volatility', 
    'max_drawdown', 'avg_drawdown', 'return/maxdd', 'return/avgdd', 
    'up_ratio', 'buy_ratio', 'flip_ratio'
]

better = met.sort_values('sharpe_ratio', ascending=False)[:'no_3vote'][show_metrics]
better

In [ ]:
better[better['sharpe_ratio']>=0.975895]

In [ ]:
ll = ['no_3vote'] + list(better.index[:10])
show_cum_ret(vote_all_kinds_return_correct_comm_levc[ll])
display_df(better.loc[ll, :].sort_values('sharpe_ratio', ascending=False), export_img=True)

In [ ]:
ll = list(better.index)

count_ll = []
for yy in range(2008, 2023):
#     show_cum_ret(vote_all_kinds_return_correct_comm_levc[ll][str(yy)])
    met = empy_metric(vote_all_kinds_return_correct_comm_levc[ll][str(yy)])

    met['up_ratio'] = vote_all_kinds[ll][str(yy)].applymap(lambda x: 0 if x<0.99 else 1).mean()
    met['buy_ratio'] = vote_all_kinds[ll][str(yy)].mean()
    met['flip_ratio'] = flip[ll][str(yy)].mean()
#     display_df(met.sort_values('sharpe_ratio', ascending=False), export_img=True)
    
    base_sharpe = met.loc['no_3vote', 'sharpe_ratio']
    met = met[met['sharpe_ratio']>=base_sharpe]
    ll = list(met.index)
    count_ll += list(met.index)
#     break

In [ ]:
count_ratio = pd.Series(count_ll).value_counts()/len(range(2008, 2023))
count_ratio.plot()

In [ ]:
display(count_ratio[count_ratio == 1])
ll = count_ratio[count_ratio == 1].index
show_cum_ret(vote_all_kinds_return_correct_comm_levc[ll])
display_df(better.loc[ll, :].sort_values('sharpe_ratio', ascending=False), export_img=True)

In [ ]:
display(count_ratio[(count_ratio>=0.5)&(count_ratio!=1)])
ll = list(count_ratio[(count_ratio>=0.5)&(count_ratio!=1)].index) + ['no_3vote']
show_cum_ret(vote_all_kinds_return_correct_comm_levc[ll])
display_df(better.loc[ll, :].sort_values('sharpe_ratio', ascending=False), export_img=True)